### Theory

#### Hypothesis Testing – The problem of multiple comparisons [5 points]

##### Question A
On the first experiment ($m = 1$) the chance that the results are significant given that the experiment lacks the power to reject is equal to $\alpha$. On the second experiment it is equal to $\alpha * P(\text{We didn't reject experiment } m-1)$. So

$$P(m^{th} \text{ experiment is significant | } m \text{ experiments lacking power to reject } H_0) = (1 - \alpha)^{m-1} * \alpha$$

Example:

$\alpha = 0.05$

$m = 6$

$P(6^{th} \text{ experiment is significant | } 6 \text{ experiments lacking power to reject } H_0) = (1-0.05)^{6-1}*0.05 \approx 0.038$

#### Question B

There is an increased chance of finding at least one false significant finding the more tests you perform. If m independent comparisons are performed, the family wise error rate (FWER) can be used. The formula for the FWER is the following:

$$\alpha_{fw} = 1-(1-\alpha)^{m}$$

so

$$P(\text{ at least one significant result | } m \text{ experiments lacking power to reject } H_0) = \alpha_{fw}$$

To arrive at this result, observe that this is $1 - P(m \text{ insignificant results | } m \text{ experiments lacking power to reject } H_0)$

Example:

$\alpha = 0.05$

$m = 6$

$\alpha_{fw} = 1-(1-0.05)^6\approx0.26$

#### Bias and unfairness in Interleaving experiments [10 points]
Balance interleaving has been shown to be biased in a number of corner cases. An example was given during the lecture with two ranked lists of length 3 being interleaved, and a randomly clicking population of users that resulted in algorithm A winning ⅔ of the time, even though in theory the percentage of wins should be 50% for both algorithms. Can you come up with a situation of two ranked lists of length 3 and a distribution of clicks over them for which Team-draft interleaving is unfair to the better algorithm?

Ranking P

| D1 N |<br>
| D2 N |<br>
| D3 R |<br>

Ranking E

| D2 N |<br>
| D3 R |<br>
| D1 N |<br>

Because D3 is the only relevant document, we'd expect it's clicks to be attributed to E more often than to P.

The set of possible interleaved rankings using Team-Draft Interleaving is as follows:

| D1 N (P) | &emsp; | D1 N (P) | &emsp; | D2 N (E) | &emsp; | D2 N (E) | <br>
| D2 N (E) | &emsp; | D2 N (E) | &emsp; | D1 N (P) | &emsp; | D1 N (P) | <br>
| D3 R (P) | &emsp; | D3 R (E) | &emsp; | D3 R (P) | &emsp; | D3 R (E) | <br>

Where D3 is in the same position in all rankings, with equal assignments to P and E. Thus, in this case, the expected number of clicks is the same for both ranking functions even though ranking E should win. This is an example of insensitivy to the quality of rankings.

Example is based on: Hofmann, K., Whiteson, S., & De Rijke, M. (2011, October). A probabilistic method for inferring preferences from clicks. In Proceedings of the 20th ACM international conference on Information and knowledge management (pp. 249-258). ACM.

### Step 1: Simulate Rankings of Relevance for E and P 

In [ ]:
import itertools
import numpy as np

# Added the numbers to facilitate easy sorting.
REL_SCORES = {'N' : 0, 'R' : 1, 'HR' : 2}

# Generate all possible rankings for both Production and Experimental systems
P = list(itertools.product(REL_SCORES.keys(), repeat = 5))
E = list(itertools.product(REL_SCORES.keys(), repeat = 5))
 
rankings = list(itertools.product(P, E))

# Make sure the right amount of combinations are made
assert len(rankings) == 3**10

# Show a sample of the generated rankings
for p, e in rankings[0::10000]:
    print(p)
    print(e)
    print()

### Step 2: Implement Evaluation Measures

In [ ]:
# quote from slide 15: "usually user only looks at very few top results : e.g. precision@3"
def precision_at_k(ranking, k):
    subset = ranking[:k]
    n_relevant = 0
    
    for result in subset:
        if REL_SCORES[result] > 0:
            n_relevant += 1
    
    return n_relevant / k

def DCG_at_k(ranking, k, optimal=False):
    subset = ranking[:k]
    
    if optimal:
        subset = sorted(subset, reverse=True, key=lambda x: REL_SCORES[x])
        
    discounted_score = 0
    
    for i, result in enumerate(subset):
        # prepare variables for DCG formula
        rank = i + 1
        rel = REL_SCORES[result]
        
        # Calculate score
        score = (2**rel - 1) / np.log2(1 + rank)
        discounted_score += score
    
    # NB: sum is part of the formula
    return discounted_score
        
def nDCG_at_k(ranking, k):
    true = DCG_at_k(ranking, k)
    best = DCG_at_k(ranking, k, optimal=True)
    
    # This is to prevent dividing by 0. If the best possible DCG is 0, the
    # true DCG is also zero (because there are no relevant document is the top k)
    if best == 0.0: 
        return 0.0
    else:
        return true / best

def prob_of_relevance(grade, max_grade):
    return (2**grade - 1) / 2**max_grade

def ERR(ranking):
    """Algorithm to compute ERR in linear time, implemented based on 
    
    Chapelle, O., Metlzer, D., Zhang, Y., & Grinspan, P. (2009, November). 
    Expected reciprocal rank for graded relevance. In Proceedings of the 18th ACM conference
    on Information and knowledge management (pp. 621-630). ACM."""
    
    grades = [REL_SCORES[x] for x in ranking]
    max_grade = max(REL_SCORES.values())
    
    p = 1
    err = 0
    
    for r in range(len(grades)):
        R = prob_of_relevance(grades[r], max_grade)
        err += p * (R/(r+1))
        p *= (1 - R)
    
    return err

In [ ]:
## This cell does te calculations for all three metrics and saves the results to a list    

depth = 3

precision_p = []
precision_e = []
dcg_p = []
dcg_e = []
err_p = []
err_e = []

for i, (p, e) in enumerate(list(rankings)):
    precision_p.append(precision_at_k(p, depth))
    precision_e.append(precision_at_k(e, depth))

    dcg_p.append(nDCG_at_k(p, depth))
    dcg_e.append(nDCG_at_k(e, depth))
    
    err_p.append(ERR(p))
    err_e.append(ERR(e))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
plt.subplot(211)
plt.plot(range(len(dcg_p)), dcg_p)

# plt.subplot(212)
# plt.plot(range(len(dcg_e)), dcg_e, linewidth=0.1)

plt.figure(2)
plt.subplot(211)
plt.plot(range(len(precision_p)), precision_p)

# plt.subplot(212)
# plt.plot(range(len(precision_e)), precision_e, linewidth=0.1)

fig = plt.figure(3)
plt.subplot(211)
plt.plot(range(len(err_p)), err_p)

# plt.subplot(212)
# plt.plot(range(len(err_e)), err_e, linewidth=0.1)

plt.show()


#TODO: asxislabels, graphtitles
#TODO: Analsyse algemener maken. De volgorde van R / N / HR is anders op elke machine

### Analysis

The lists in the `rankings` variable represents a simulation of all possible outcomes of a comparisson of model P and model E on a query. Because of the way the combinations are generated, subsequent instances of the E model vary rapidly and thus produce chaotic graphs. So for the graphical analysis, we will focus on model P.

The first thing that stands out is that there is no obvious pattern in the progression of the metrics as you iterate trough the instances. We would've expected a more symmectric shape of the plot. But at further inspection, there is indeed a pattern. This is especially viable when you compare the three metrics with eachoter. All metrics are mininum around the 30.000th pair, which where the rankings are mostly non-relevant. At the beginning and end of the simulations the rankings are made up of mostly 'R' and 'HR' documents so they score high. In the EGG chart you get a more detailed view, since the grade of the rank is taken into account and grade('R') < grade('HR').

### Step 3: Calculate the 𝛥measure

In [ ]:
# In the assignment is says to only consider pairs for which E > P. But we found it easier to keep
# all pairs, because we have to compare the results with the online rankings. If we discard some pairs,
# the arrays don't line up anymore.

def delta_measure(scores_a, scores_b):
    return [b - a for a, b in zip(scores_a, scores_b)]

def percentage_of_wins_for_E(delta_values):
    return np.sum(np.array(delta_values) > 0) / len(delta_values)

# Calculate delta measure for the three chosen metrics
delta_precision = delta_measure(precision_p, precision_e)
delta_nDCG = delta_measure(dcg_p, dcg_e)
delta_ERR = delta_measure(err_p, err_e)

print("Delta precision")
print()
print(delta_precision[0::1000])
print()
print("Percentage of wins for E:", percentage_of_wins_for_E(delta_precision))
print()
print()
print("Delta nDCG")
print()
print(delta_nDCG[0::1000])
print()
print("Percentage of wins for E:", percentage_of_wins_for_E(delta_nDCG))
print()
print()
print("Delta ERR")
print()
print(delta_ERR[0::1000])
print()
print("Percentage of wins for E:", percentage_of_wins_for_E(delta_ERR))

### Step 4:  Implement Interleaving

In [ ]:
import random
# Step 4: Implement 2 interleaving algorithms: 
## (1) Team-Draft Interleaving OR Balanced Interleaving, AND 
## (2) Probabilistic Interleaving.

# The interleaving algorithms should: 
## (a) given two rankings of relevance interleave them into a single ranking, and 
## (b) given the users clicks on the interleaved ranking assign credit to the algorithms 
##     that produced the rankings.

# (Note 4: 
## Note here that as opposed to a normal interleaving experiment where rankings consists of urls or docids, 
## in our case the rankings consist of relevance labels. Hence in this case 
## (a) you will assume that E and P return different documents, 
## (b) the interleaved ranking will also be a ranking of labels.)


### BALANCED INTERLEAVING:
# Find two random rankings A and B and present them:
#A, B = ['a','b','c','d','g','h'], ['b','e','a','f','g','h']
A, B = random.choice(rankings)
A, B = [str(a) + "P" + str(i) for i, a in enumerate(A)], [str(b) + "E" + str(i) for i, b in enumerate(B)]

# initialize pointers p_a and p_b
p_a, p_b, I = 0, 0, []

# Flip a coin to decide which pointer to pick highest value from ranking.
turn = 'A' if random.randint(0, 1) == 0 else 'B'

# Greedily collect rankings from both lists and build the interleaved list.
print(turn, "is first:")
while p_a < len(A) or p_b < len(B):
    if p_a < p_b or p_a == p_b and turn == 'A':
        if A[p_a] not in I:
            I.append(A[p_a])
        p_a += 1
    else:
        if B[p_b] not in I:
            I.append(B[p_b])
        p_b += 1

print(I)

In [ ]:
### TEAM-DRAFT INTERLEAVING:


# Fully based on example in paper: TODO
#A, B = ['a','b','c','d','g','h'], ['b','e','a','f','g','h']
A, B = random.choice(rankings)
#print(A, B)

A, B = [str(a) + "P" + str(i) for i, a in enumerate(A)], [str(b) + "E" + str(i) for i, b in enumerate(B)]

# create teams
team_a, team_b, I = 0, 0, []

while any(set(A)-set(I)) or any(set(B)-set(I)):
    turn = 'A' if random.randint(0, 1) == 0 else 'B'
    if team_a < team_b or team_a == team_b and turn == 'A':
        for a in A:
            if a not in I:
                I.append(a)
                break
        team_a += 1
    else:
        for b in B:
            if b not in I:
                I.append(b)
                break
        team_b += 1

def team_draft_interleaving(A,B):
    # create teams
    team_a, team_b, I = 0, 0, []
    
    # While there are still documents not present in the interleaved list
    while any(set(A)-set(I)) or any(set(B)-set(I)):
        
        #Flip a coin
        turn = 'A' if random.randint(0, 1) == 0 else 'B'
        
        #If A wins the coin flip: put the highest ranked document of A
        # that is not yet in the interleaved list, in the interleaved
        if team_a < team_b or team_a == team_b and turn == 'A':
            for a in A:
                if a not in I:
                    I.append(a)
                    break
            team_a += 1
        # If B wins the flip: put the highest ranked document of A
        # that is not yet in the interleaved list, in the interleaved
        else:
            for b in B:
                if b not in I:
                    I.append(b)
                    break
            team_b += 1
    return(I)



interleaved_lists = []

for A,B in rankings:
    A, B = [str(a) + "P" + str(i) for i, a in enumerate(A)], [str(b) + "E" + str(i) for i, b in enumerate(B)]
    interleaved_list = team_draft_interleaving(A,B)
    interleaved_lists.append(interleaved_list)

print(interleaved_lists[:1000])

### Step 5:  Implement User Clicks Simulation

In [92]:
''' In this cell the Random click model is implemented. We use the PyClick module to parse the Yandex file.
Our TA assured us it is fine, if we use it just for the parsing of the Yandex file.
Furthermore we assume all queries should be used, even if none of the results are clicked'''

def execute_clicks(click_probs):
    """Returns a list of booleans indicating on which positions was clicked, based on the
    given click_probs (click probabilities)"""
    
    # Flip one coin for (and with) every probability in click_probs
    return np.random.binomial(1, p=click_probs)

def MLE_random_click(sessions):
    '''As slide 62 of lecture 3 shows, the MLE of a bernouilli distribution, rho, is just
    the ratio of clicks with respect to the total number of URls shown. 
    '''
    total_clicks = 0
    total_shown = len(sessions)*10
    
    for sesh in sessions:
        total_clicks += sum(sesh.get_clicks())
    
    rho = total_clicks/total_shown
    return rho
            
def random_click_probabilities(ranking, rho):
    '''probabilities of a click from the random clicking model'''
    return [rho] * len(ranking)

def simulate_random_click_model(ranking, rho, N):
    '''Generate N click patterns based on the random click model'''
    return [execute_clicks(random_click_probabilities(ranking, rho)) for _ in range(N)]

def sdbn_click_probabilities(ranking, alphas, sigmas):
    '''Probabilities of a click from the Simple Bayesian Dynamic Network'''
    
    # Assume zero probabilities
    click_probs = [0.0] * len(ranking)
    
    for i, relevance in enumerate(ranking):
        
        # Click when attracted
        alpha = alphas[relevance[:-2]]
        click_probs[i] = alpha
        
        # If satisfied, we do not evaluate the rest of the documents
        satisfied = float(np.random.choice([0,1], 1, p=[(1-sigmas[i]),(sigmas[i])]))
        if satisfied:
            break
    
    return click_probs

def simulate_sdbn_click_model(ranking, alphas, sigmas, N):
    '''Generate N click patterns based on the SDBN click model'''
    return [execute_clicks(sdbn_click_probabilities(ranking, alphas, sigmas)) for _ in range(N)]

In [ ]:
import pyclick
from pyclick.utils import YandexRelPredChallengeParser

parser = YandexRelPredChallengeParser.YandexRelPredChallengeParser()
search_sessions = parser.parse('YandexRelPredChallenge.txt')

In [ ]:
# Assumption: 1 query always returns the same docIDs (ie. ranking is deterministic)
# TODO: dit kunnen we in principe gewoon testen

from collections import Counter

def learn_sdbn_satisfaction(search_sessions):
    last_click_ranks = dict()
    clicks_per_query = dict()
    last_total_ratio = dict()

    # Find last click rank per query:
    for session in search_sessions:
        
        # If get_last_click_rank returns 10, there hasn't been click action for the query.
        if session.get_last_click_rank() == 10:
            continue

        # Find last click:
        if session.query in last_click_ranks:
            last_click_ranks[session.query].append(session.get_last_click_rank())
        else:
            last_click_ranks[session.query] = [session.get_last_click_rank()]
            
        # Find all click ranks per query:
        if session.query not in clicks_per_query:
            clicks_per_query[session.query] = []

        for idx, val in enumerate(session.get_clicks()):
            if val == 1:
                clicks_per_query[session.query].append(idx)    

    # Calculate sigmas for each unique query and each of its ranks:
    for query in last_click_ranks.keys():
        last_total_ratio[query] = [0] * 10
    
        count_last = Counter(last_click_ranks[query])
        count_total = Counter(clicks_per_query[query])
    
        for rank in list(count_last):
            sigma = count_last[rank] / count_total[rank]
            
            last_total_ratio[query][rank] = sigma
            
    # Compose final list of average sigmas, i.e. 1 sigma for each rank:
    avg_sigmas = [0.0] * 10

    for query in list(last_total_ratio):
        avg_sigmas = [sum(sigma) for sigma in zip(last_total_ratio[query], avg_sigmas)]
    
    click_model = [sigma / len(last_total_ratio) for sigma in avg_sigmas]
    norm_click_model = [float(sigma) / sum(click_model) for sigma in click_model]
    
    return last_click_ranks, clicks_per_query, last_total_ratio, click_model, norm_click_model


%time last, total, sigmas, cm, ncm = learn_sdbn_satisfaction(search_sessions)
# np.sum(sigmas, axis=1)

In [ ]:
print(sorted(last["209"]))
print()
print(sorted(total["209"]))
print()
print(sigmas["209"])

ncm

### Step 6: Simulate Interleaving Experiment

In [89]:
'''Random click model experiment'''

def click_ranker_mapping(clicks, interleaved_list):
    '''Input a list of clicks (1 or 0) and a interleaved list with information on which ranker
    attributed which file.
    
    Output the winner of the interleaved list if any. Else it outputs a tie.'''
    
    # counters for how many times P and E are clicked
    P_click = 0
    E_click = 0
    
    for i, value in enumerate(clicks):
        #if clicked
        if value == 1.0:
            # Check which algorithm contributed it to the list
            if interleaved_list[i][-2] == 'E':
                E_click += 1
            else:
                P_click += 1
                    
    if P_click > E_click:
        return 'P'
    if E_click > P_click:
        return 'E'
    
    return 'Tie'

def experiment(clicks, interleaved_list):
    '''Does experiments for one interleaved list with a list of click patterns.
    Outputs a proportion of wins for E over all experiments, ties included.'''
    
    clicks = np.array(clicks)
    
    # Boolean array where each contribution of E is marked True
    E_docs = np.array([x[-2] == 'E' for x in interleaved_list])
    
    # Overlay of clicks and the boolean mask above. Evaluates
    # to True if a rank has been clicked and was contributed by E.
    # Finally sum those Truths (1s) to get total clicks on E per experiment
    E_clicks = np.sum(np.logical_and(E_docs, clicks), axis=1)
    
    # Clicks on P are the difference of total clicks and E-clicks
    P_clicks = np.sum(clicks, axis=1) - E_clicks
    
    return  np.sum(E_clicks > P_clicks) / len(clicks)

def experiment_loop(clicks, interleaved_list):
    '''Does N experiments for one interleaved list
    Outputs a proportion of wins for E over all experiments, ties included.'''
    
    P_wins = 0
    E_wins = 0
    
    for click_pattern in clicks:
        winner = click_ranker_mapping(click_pattern, interleaved_list)
        
        if winner == 'P':
            P_wins += 1
        
        if winner == 'E':
            E_wins += 1
    
    return (E_wins/len(clicks))

In [91]:
'''In this cell the experiment for the random click model is run'''

E_win_proportions = {
    'random' : [],
    'sdbn' : []
}

# Estimate alpha_uq from relevance labels

# Katja Hofmann, Anne Schuth, Shimon Whiteson, and Maarten de Rijke. Reusing historical interaction
# data for faster online learning to rank for IR. In WSDM, 2013. ACM Press. 69, 70, 77

alpha = {
    'perfect' : {'N' : 0.0, 'R' : 0.5, 'HR' : 1.0},
    'navigational' : {'N' : 0.05, 'R' : 0.5, 'HR' : 0.95},
    'informal' : {'N' : 0.4, 'R' : 0.7 , 'HR' : 0.9}
}

# Calculate the Rho for the random click model
rho = MLE_random_click(search_sessions)

import time

N = 100
start = time.time()
for i in range(len(interleaved_lists)):
    if i % 100 == 0 and i > 0:
        print("{} / {} \t\t at {}/sec".format(i+1, len(interleaved_lists), (i+1)/(time.time() - start)))
    
    # Random Click Model simulation
    random_clicks = simulate_random_click_model(interleaved_lists[i], rho, N)
#     e1 = experiment(random_clicks, interleaved_lists[i])
#     e2 = experiment_loop(random_clicks, interleaved_lists[i])
#     assert e1 == e2, "{} / {}".format(e1, e2) 
    E_win_proportions['random'].append(experiment(random_clicks, interleaved_lists[i]))
    
    # SDBN simulation
    sdbn_clicks = simulate_sdbn_click_model(interleaved_lists[i], alpha['navigational'], ncm, N)
    E_win_proportions['sdbn'].append(experiment(sdbn_clicks, interleaved_lists[i]))

101 / 59049 		 at 17.862511797488374/sec
201 / 59049 		 at 17.40684748053797/sec


KeyboardInterrupt: 

### Step 7: Analysis

In [ ]:
'''In this cell the visualizations for the random click model experiment are generated


#Beautiful

'''

# Tournament: when does P beat E and does it depend on the metric?
import random
import scipy.stats as stats

def mean_scores(metric, sample):
    result_p = []
    result_e = []

    for p, e in sample:

        result_p.append(metric(p))
        result_e.append(metric(e))
    
    return result_p, result_e

def test_greater_than(sample_a, sample_b, alpha):
    """Tests whether the difference [mean(sample_a) - mean(sample_b)] is 
    significantly greater than zero"""
    
    t_test = stats.ttest_ind(sample_a, sample_b)
    return t_test.statistic > 0 and t_test.pvalue/2 < alpha

def test_greater_than_single_sample(sample, alpha):
    
    t_test = stats.ttest_1samp(sample, 0.5)
    return t_test.statistic > 0 and t_test.pvalue/2 < alpha


In [ ]:
# Possible test:
# - Is there a meaningful difference between SDBN and Random
# - Is there a meaningful difference between 

wins_under_precision = [delta > 0 for delta in delta_precision]
wins_under_nDCG = [delta > 0 for delta in delta_nDCG]
wins_under_ERR = [delta > 0 for delta in delta_ERR]
wins_under_SDBN = [p > 0.5 for p in E_win_proportions['sdbn']]
wins_under_RCM = [p > 0.5 for p in E_win_proportions['random']]

print("Interleaving:")
print("Mean proportion of wins for E under sdbn", np.mean(E_win_proportions['sdbn']), test_greater_than_single_sample(E_win_proportions['sdbn'], 0.05))